# HyperDual Numbers (first derivatives)

The idea of Dual numbers can be applied to multiple ($n$) number of variables, through the Taylor expansion
$$
f(\vec x + \vec \epsilon) \approx f(\vec x) + \sum_{i=0}^{n-1} \ \epsilon_i \ \partial_i f(\vec x) \; ,
$$
which also allows us to define the derivative as 
$$
\partial_i f(\vec x) = \dfrac{\partial f(\vec x + \vec \epsilon)}{ \partial \epsilon_i} \Big|_{\vec\epsilon=0} \;.
$$

In [1]:
import numpy as np

In [2]:
class HyperDualNumber:
    def __init__(self, real, dual):
        '''
        real: the real part
        dual: the dual parts
        '''
        self.real = real
        self.dual = dual[:]
        self.NVar=len(dual)

    def __neg__(self):
        return  HyperDualNumber(-self.real,[-_ for _ in self.dual])

    def __add__(self,d):
        
        if type(d) is HyperDualNumber:
            return HyperDualNumber(self.real+d.real,[self.dual[i]+d.dual[i] for i in range(self.NVar)] )
        else:
            return HyperDualNumber(self.real+d,self.dual[:])
                
    def __sub__(self,d):
        if type(d) is HyperDualNumber:
            return HyperDualNumber(self.real-d.real,[self.dual[i]-d.dual[i] for i in range(self.NVar)])
        else:
            return HyperDualNumber(self.real-d,self.dual[:])
    
    def __mul__(self,d):
        if type(d) is HyperDualNumber:
            return HyperDualNumber(self.real*d.real,[d.real*self.dual[i]+self.real*d.dual[i] for i in range(self.NVar)])
        else:
            return HyperDualNumber(self.real*d,self.dual[:])
        
    def __truediv__(self,d):
        if type(d) is HyperDualNumber:
            return HyperDualNumber(self.real/d.real, [(self.dual[i]-self.real*d.dual[i]/d.real)/d.real for i in range(self.NVar)] )
        else:
            return HyperDualNumber(self.real/d,self.dual[i])
   
        
    def __repr__(self):
        return "({0},{1})".format(self.real,self.dual)
    
    def __pow__(self,n):
        return HyperDualNumber(self.real**n, [n*self.real**(n-1)*self.dual[i] for i in range(self.NVar)]  )

    
    def getDer(self,i):
        return self.dual[i]


In [3]:
#define a few basic functions
def sin(d):
    if type(d) is HyperDualNumber:
        return HyperDualNumber(np.sin(d.real),[np.cos(d.real)*d.dual[i] for i in range(d.NVar)])
    else:
        return HyperDualNumber(np.sin(d),np.cos(d))

def cos(d):
    if type(d) is HyperDualNumber:
        return HyperDualNumber(np.cos(d.real),[-np.sin(d.real)*d.dual[i] for i in range(d.NVar)])
    else:
        return HyperDualNumber(np.cos(d),-np.sin(d))

def exp(d):
    if type(d) is HyperDualNumber:
        return HyperDualNumber(np.exp(d.real),[np.exp(d.real)*d.dual[i] for i in range(d.NVar)])
    else:
        return HyperDualNumber(np.exp(d.real),np.exp(d.real))

def log(d):
    if type(d) is HyperDualNumber:
        return HyperDualNumber(np.log(d.real),[d.real**-1*d.dual[i] for i in range(d.NVar)])
    else:
        return HyperDualNumber(np.log(d.real),d.real**-1)

def pow(d,n):
    if type(d) is HyperDualNumber and (not type(n) is HyperDualNumber):
        return d**n
    
    if (not type(d) is HyperDualNumber) and (type(n) is HyperDualNumber):
        return  HyperDualNumber(d**n.real,[d**n.real*(np.log(d) * n.dual[i]) for i in range(n.NVar)]  )

    if (type(d) is HyperDualNumber) and (type(n) is HyperDualNumber):
        return  HyperDualNumber(d.real**n.real,[d.real**n.real*(n.real/d.real * d.dual[i] + np.log(d.real) * n.dual[i]) for i in range(n.NVar)]  )


In [4]:
const=HyperDualNumber(1,[0,0])#a constant

d1=HyperDualNumber(2,[1,0])#variable x_0
d2=HyperDualNumber(5,[0,1])#variable x_1

In [5]:
print(d1+d2,d1*d2,d1/d2)

(7,[1, 1]) (10,[5, 2]) (0.4,[0.2, -0.08])


In [6]:
print(d1**2,d1*d1)

(4,[4, 0]) (4,[4, 0])


In [7]:
pow(d1,d2)

(32,[80.0, 22.18070977791825])

In [8]:
sin(d1)*d2,cos(d2*exp(d1))

((4.546487134128409,[-2.080734182735712, 0.9092974268256817]),
 (0.7290719093657502,[25.286720832541537, 5.057344166508307]))

In [9]:
exp(d1)+d2,log(d1+d2)

((12.38905609893065,[7.38905609893065, 1.0]),
 (1.9459101490553132,[0.14285714285714285, 0.14285714285714285]))